In [1]:
START_DATE = "1970-01-01"
END_DATE = "2023-12-31"

In [2]:
import ee
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
# Autentica com o Google Earth Engine
# Descomente se for a primeira execução
# ee.Authenticate()


In [5]:

# Inicializa o Earth Engine
ee.Initialize(project=os.environ.get("GOOGLE_EARTH_ENGINE_PROJECT_ID"))


In [6]:
# Carrega geometria de Minas Gerais (FAO GAUL, nível estadual)
mg = (
    ee.FeatureCollection("FAO/GAUL/2015/level1")
    .filter(ee.Filter.eq("ADM1_NAME", "Minas Gerais"))
    .filter(ee.Filter.eq("ADM0_NAME", "Brazil"))
    .geometry()
)

In [7]:
# Seleciona a coleção NDVI do MODIS (250m, 16 dias)
ndvi_collection = ee.ImageCollection("MODIS/006/MOD13Q1") \
    .filterBounds(mg) \
    .filterDate(START_DATE, END_DATE) \
    .select('NDVI')

/home/bruno/anaconda3/envs/fiap/lib/python3.11/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


In [8]:
# Função para reduzir por média regional e transformar em Feature com data e valor
def image_to_feature(img):
    mean_dict = img.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=mg, scale=250, maxPixels=1e13
    )
    return ee.Feature(
        None, {"date": img.date().format("YYYY-MM-dd"), "ndvi": mean_dict.get("NDVI")}
    )

In [9]:
# Aplica a função a todas as imagens
ndvi_features = ndvi_collection.map(image_to_feature)

In [10]:
# Coleta os dados em Python
ndvi_list = ndvi_features.getInfo()['features']

In [14]:
import pandas as pd  # Converte para DataFrame


data = [
    {"date": f["properties"]["date"], "ndvi": f["properties"]["ndvi"]}
    for f in ndvi_list
]
df = pd.DataFrame(data)

df['ndvi'] = pd.to_numeric(df['ndvi'], errors='coerce') / 10000  # escala 0–1

df

,date,ndvi
0,2000-02-18,0.541287
1,2000-03-05,0.650413
2,2000-03-21,0.669972
3,2000-04-06,0.679860
4,2000-04-22,0.655252
...,...,...
524,2022-12-03,0.636841
525,2022-12-19,0.603338
526,2023-01-01,0.643354
527,2023-01-17,0.713500


In [12]:
# Exporta como CSV
if not os.path.exists("../data/GOOGLE_EARTH_ENGINE"):
    os.makedirs("../data/GOOGLE_EARTH_ENGINE")
df.to_csv("../data/GOOGLE_EARTH_ENGINE/ndvi_minas_gerais.csv", index=False)
print("✅ CSV salvo com sucesso!")

✅ CSV salvo com sucesso!
